# Sentinel-2 <img align="right" src="../Supplementary_data/DE_Africa_Logo_Stacked_RGB_small.jpg">

* **Products used:** 
[s2_l2a](https://explorer.digitalearth.africa/s2_l2a)


## Description

1. This notebook is used to download Sentinel-2 imagery for sites where Arable Mark sensors are installed, using a 0.0025 degree rectangular buffer.
2. The script requires a .csv file with site locations and dates, "DEA_site_seasons.csv"
3. S2 images are downloaded to the 'dea_export_s2' folder, which is then converted to a zip archive.
4. You must then maually download the zip archive.



***

### Load packages

In [1]:
import datacube

from deafrica_tools.datahandling import load_ard
from deafrica_tools.plotting import rgb

### Connect to the datacube

In [2]:
%matplotlib inline

import datacube
from datacube.utils.cog import write_cog

from deafrica_tools.datahandling import load_ard
from deafrica_tools.plotting import rgb
import pandas as pd

In [3]:
dc = datacube.Datacube(app='Exporting_GeoTIFFs')

In [4]:

dea_sites = pd.read_csv("/home/jovyan/Datasets/DEA_site_seasons.csv")
dea_sites.head(5)

,Location,new_start,new_end,X,lon,lat,Pod1,Pod2,Pod3,Group,start_date,end_date,Ortho,Scene,order_id_s_h,Valid_season?,Multiple_seasons?
0,choma_A000127,2017-09-02,2018-03-02,22,27.0922,-16.7795,A000127,NaN,NaN,choma,11/01/2017,05/31/2018,0,85,0d688599-b6bb-47d8-a3cd-a6a8d736ec5b,NaN,NaN
1,choma_A000127,2018-03-02,2018-07-30,22,27.0922,-16.7795,A000127,NaN,NaN,choma,11/01/2017,05/31/2018,0,85,0d688599-b6bb-47d8-a3cd-a6a8d736ec5b,NaN,NaN
2,choma_A000212,2017-09-02,2018-03-02,23,26.9253,-16.8294,A000212,NaN,NaN,choma,11/01/2017,05/31/2018,0,2,97cfdbe9-39d1-41ab-8e6e-3d355b598b85,NaN,NaN
3,choma_A000212,2018-03-02,2018-07-30,23,26.9253,-16.8294,A000212,NaN,NaN,choma,11/01/2017,05/31/2018,0,2,97cfdbe9-39d1-41ab-8e6e-3d355b598b85,NaN,NaN
4,choma_A000247,2017-09-02,2018-03-02,24,27.1179,-16.7724,A000247,NaN,NaN,choma,11/01/2017,05/31/2018,0,76,95c69caa-67fa-405f-ae2d-0456838c7272,NaN,NaN


Query and save S2 files

In [ ]:
for k in range(2):
    print(k)
    site_name = dea_sites.loc[k]['Location']
    print(site_name)
    xmin = dea_sites.loc[k]['lon'] - 0.0025
    xmax = dea_sites.loc[k]['lon'] + 0.0025
    ymin = dea_sites.loc[k]['lat'] - 0.0025
    ymax = dea_sites.loc[k]['lat'] + 0.0025
    date_min = dea_sites.loc[k]['new_start']
    date_max = dea_sites.loc[k]['new_end']

    query = {
        'x': (xmin, xmax),
        'y': (ymin, ymax),
        'time': (date_min, date_max),
        'resolution': (-10, 10),
 #   'measurements': ['red', 'green', 'blue'],
        'output_crs':'EPSG:6933'
    }
    print(query)
    
    ds = load_ard(dc=dc,
              products=['s2_l2a'],
              **query)
# Load available data from Landsat 8 and filter to retain only times
# with at least 50% good data
    print(str(len(ds.time)) +  " images")

    for i in range(len(ds.time)):
        
    # We will use the date of the satellite image to name the GeoTIFF
        date = ds.isel(time=i).time.dt.strftime('%Y-%m-%d').data
        print(i)
        print(f'Writing {date}')

    # Convert current time step into a `xarray.DataArray`
        singleTimestamp = ds.isel(time=i).to_array()
        write_cog(singleTimestamp,
              fname= "/home/jovyan/Datasets/dea_export_s2/" + site_name + "_" + f'{date}.tif',
              overwrite=True)




In [ ]:
print(ds.variables)
da = ds.to_array()
da

Create zip archive of S2 imagery

In [12]:
import shutil
shutil.make_archive("dea_s2", 'zip', "/home/jovyan/Datasets/dea_export_s2")
#shutil.make_archive("dea_s1", 'zip', "/home/jovyan/Datasets/dea_export_s1")

'/home/jovyan/Datasets/dea_s1.zip'

After archive is created, manually download zip 